# Perception Test 
- Created 1 April 2020

In [28]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import os
from os import makedirs
from os.path import join
import scipy

import librosa

## Load Speech & Label 

In [40]:
def load_data(record_path, resyn_path):
    
    with open(join(record_path,'sound_set.txt'), 'r') as f:
        data = np.array(f.read().split(','))
        
    audio_data = np.array([librosa.load(join(resyn_path, 'sound{}.wav'.format(file)), sr=16000)[0] for file in range(81)])
    with open(join(record_path,'syllable_name.txt'), 'r') as f:
        labels = np.array(f.read().split(','))

    return audio_data, labels

In [35]:
audio_data, labels = load_data(RECORD_DATA, RESYN_DATA)

In [36]:
print(audio_data.shape)
print(labels.shape)

(162,)
(162,)


## Shuffle Data

In [21]:
def shuffle(audio_data, labels):

    shuffle_idx = np.random.permutation(audio_data.shape[0])

    audio_data_shuff = np.array([audio_data[i] for i in shuffle_idx])
    labels_shuff = np.array([labels[i] for i in shuffle_idx])

    return audio_data_shuff, labels_shuff, shuffle_idx

## Created directory and export data

In [64]:
def create_perception_set(audio_data, labels, idx):

    folder = 'perception_set{}'.format(idx)
    target_folder = 'target'
    label = 'ans_set{}.csv'.format(idx)
    form = 'form_set{}.csv'.format(idx)

    audio_data, labels, _ = shuffle(audio_data, labels)
    
    makedirs(folder, exist_ok=True)
    makedirs(target_folder, exist_ok=True)
    pd.DataFrame(labels).to_csv(join(target_folder, label), header=None, index=None)
    _ = [scipy.io.wavfile.write(join(folder,'sound{}.wav'.format(idx)), 16000, sig) for idx, sig in enumerate(audio_data)]
    pd.DataFrame(np.array([['sound{}.wav'.format(idx),''] for idx, sig in enumerate(audio_data)]),
                columns=['FILENAME', 'ANSWER']).to_csv(join(folder, form), index=None)

In [65]:
create_perception_set(audio_data, labels, 1)

## Run 

In [66]:
RECORD_DATA = '../../data/d_records/d_record_set_2/'
RESYN_DATA = '../../experiment_2/result/predict_129_2/sound/'
SET = 25

In [67]:
audio_data, labels = load_data(RECORD_DATA, RESYN_DATA)

In [ ]:
for idx in range(SET):
    create_perception_set(audio_data, labels, idx)

# Perception Analysis

In [106]:
def transform_map(df, col=0):
    df[col] = df[col].str.strip()
    df[col] = df[col].str.replace('โอ','o')
    df[col] = df[col].str.replace('ออ','O')
    df[col] = df[col].str.replace('อา','a')
    df[col] = df[col].str.replace('อี','i')
    df[col] = df[col].str.replace('อู','u')
    df[col] = df[col].str.replace('เอ','e')
    df[col] = df[col].str.replace('แอ','E')
    df[col] = df[col].str.replace('อือ','M')
    df[col] = df[col].str.replace('เออ','7')
    return df

def inverse_transform_map(df, col=0):
    df[col] = df[col].str.strip()
    df[col] = df[col].str.replace('o','โอ')
    df[col] = df[col].str.replace('O','ออ')
    df[col] = df[col].str.replace('a','อา')
    df[col] = df[col].str.replace('i','อี')
    df[col] = df[col].str.replace('u','อู')
    df[col] = df[col].str.replace('e','เอ')
    df[col] = df[col].str.replace('E','แอ')
    df[col] = df[col].str.replace('M','อือ')
    df[col] = df[col].str.replace('7','เออ')
    return df

In [68]:
ANS_FILE = 'result'
TARGET_FILE = 'target'
SET = 3

In [121]:
ans = pd.concat((pd.read_csv(join(ANS_FILE, 'form_set{}.csv'.format(idx))) for idx in range(SET))).reset_index(drop=True)
target = pd.concat((pd.read_csv(join(TARGET_FILE, 'ans_set{}.csv'.format(idx)), header=None) for idx in range(SET))).reset_index(drop=True)

In [122]:
ans_col = 'ANSWER'
target_col = 0
target = transform_map(target, col=target_col)
ans = transform_map(ans, col=ans_col)

In [123]:
data = np.concatenate([np.array([[i,idx, 1] for item in target[target_col].values.tolist() for idx,i in enumerate(item)]),np.array([ [i,idx, 0] for item in ans[ans_col].values.tolist() for idx, i in enumerate(item)])], axis=0)
result = pd.DataFrame(data,columns=['phonetic', 'position', 'target'])
result.to_csv('result.csv', index=None)

In [124]:
result.head()

,phonetic,position,target
0,u,0,1
1,M,1,1
2,E,0,1
3,M,1,1
4,O,0,1
